In [263]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

import time

In [264]:
# Путь к сводной таблице
path_to_data = 'data/СВОД (7).xlsx'

In [265]:
# Создаем список обрабатываемых листов
lst_sheets = ['Свод_2019','Свод_2020','Свод_2021','Свод_2022']

In [274]:
# Открываем поочередно
for sheet in lst_sheets:
    year = sheet.split('_')[1]
    # Получаем список регионов на листе
    df = pd.read_excel(path_to_data,sheet_name=sheet)
    # Создаем датафрейм пропуская верхнюю строку
    te_df = pd.read_excel(path_to_data,sheet_name=sheet,skiprows=1)
    # получаем список регионов
    lst_regions = [region for region in df.columns if 'Unnamed' not in region]
    # Очищаем от лишних строк и колонок
    te_df.drop(0,axis=0,inplace=True)
    te_df.drop(columns=['№ показателя','Единица измерения','№ строки'],axis=1,inplace=True)
    te_df.drop(columns=[column for column in te_df.columns if 'только за 4-й квартал!' in column],inplace=True)
    te_df.drop(columns=[column for column in te_df.columns if 'от года создания ЦОПП' in column],inplace=True)
    # Транспонируем
    lin_df = te_df.transpose()

    # Делаем список утраивая подряд названия регионов
    lst_triple = [None]
    for region in lst_regions:
        for  i in range(2):
            lst_triple.append(region)
    
    # Вставляем в датафрейм колонку с названием субъекта
    lin_df.insert(0,'Наименование субъекта РФ',lst_triple)

    # Создаем объект openpyxl
    wb = openpyxl.Workbook()
    # Получаем список без первой колонки
    lst_col = list(lin_df.columns)[1:]
    for idx,column in enumerate(lst_col):
        temp = lin_df[column].to_frame().transpose()

        # Создаем базовый датафрейм
        base_df = temp.iloc[0,:3].to_frame().transpose()        
        for i in range(3,temp.shape[1],2):
            t_df = temp.iloc[0,i:i+2].to_frame().transpose()
            t_df.columns = ['Планируемое значение в 2022 году','Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов)']
            t_df.insert(0,'Наименование индикатора/показателя',temp.iloc[0,0])
            base_df = pd.concat([base_df,t_df],ignore_index=True,axis=0)
            # Создаем лист
        base_df.insert(1,'Фактическое значение на конец 2021 года',None)
        base_df.insert(1,'Регион',lst_regions)
        base_df.insert(1,'Год создания',year)
        
        wb.create_sheet(title=str(column),index=idx)
        for row in dataframe_to_rows(base_df, index=False, header=True):
            wb[str(column)].append(row)
    
    # Получаем текущее время
    current_time = time.strftime('%H_%M_%S')
    #Сохраняем файл
    wb.save(f'{sheet} от {current_time}.xlsx')

    




In [271]:
# Объединяем в одно
wb_2019 = openpyxl.load_workbook('Свод_2019 от 13_16_13.xlsx')
wb_2020 = openpyxl.load_workbook('Свод_2020 от 13_16_14.xlsx')

In [273]:
set(wb_2019.sheetnames) -set(wb_2020.sheetnames)

{'56'}